In [1]:
import pandas as pd
import numpy as np

TIME_FORMAT = "%Y%m%d_%H%M%S"

df = pd.read_csv("/Users/duncan/research/active_projects/reczilla/results/results.csv", sep=";")

/Users/duncan/miniconda3/envs/recsys/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (13,14,17,31,32,38,40,41,42,47) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
# slice by experiment name
df_expt = df.loc[df["experiment_name"].str.startswith("full-experiment-"), :]

df_expt.head()

,alg_name,alg_seed,cutoff_list,dataset_name,exception,experiment_name,hyperparameters_source,num_samples,original_split_path,param_alpha,...,test_metric_USERS_IN_GT_cut_5,test_metric_USERS_IN_GT_cut_50,test_metric_USERS_IN_GT_cut_6,test_metric_USERS_IN_GT_cut_7,test_metric_USERS_IN_GT_cut_8,test_metric_USERS_IN_GT_cut_9,time,time_on_test,time_on_train,time_on_val
4,UserKNNCF_asymmetric,0,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 15, 20, 30, 40...",MovieTweetingsReader,NaN,full-experiment-knn-146,default,100,gs://reczilla-results/dataset-splits/splits-v3...,NaN,...,1.0,1.0,1.0,1.0,1.0,1.0,20220417_233953,114.638383,11.768501,115.430055
5,UserKNNCF_asymmetric,0,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 15, 20, 30, 40...",MovieTweetingsReader,NaN,full-experiment-knn-146,random_0,100,gs://reczilla-results/dataset-splits/splits-v3...,NaN,...,1.0,1.0,1.0,1.0,1.0,1.0,20220417_233953,124.220636,13.444535,123.385043
6,UserKNNCF_asymmetric,0,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 15, 20, 30, 40...",MovieTweetingsReader,NaN,full-experiment-knn-146,random_1,100,gs://reczilla-results/dataset-splits/splits-v3...,NaN,...,1.0,1.0,1.0,1.0,1.0,1.0,20220417_233953,115.785270,11.932769,115.417125
7,UserKNNCF_asymmetric,0,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 15, 20, 30, 40...",MovieTweetingsReader,NaN,full-experiment-knn-146,random_2,100,gs://reczilla-results/dataset-splits/splits-v3...,NaN,...,1.0,1.0,1.0,1.0,1.0,1.0,20220417_233953,123.379156,12.692332,123.481905
8,UserKNNCF_asymmetric,0,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 15, 20, 30, 40...",MovieTweetingsReader,NaN,full-experiment-knn-146,random_3,100,gs://reczilla-results/dataset-splits/splits-v3...,NaN,...,1.0,1.0,1.0,1.0,1.0,1.0,20220417_233953,123.355272,14.126628,123.627794


In [3]:

# how many experiments, and how many samples per experiment?
# print(df_expt.groupby("experiment_name").size().rename("num_rows").reset_index())
print(df_expt.groupby(["alg_name", "dataset_name"]).size().rename("num_rows").reset_index())

                       alg_name               dataset_name  num_rows
0                  CoClustering                AnimeReader       100
1                  CoClustering         BookCrossingReader       100
2                  CoClustering              CiaoDVDReader       100
3                  CoClustering               DatingReader       100
4                  CoClustering             EpinionsReader       100
5                  CoClustering            FilmTrustReader       100
6                  CoClustering               FrappeReader       100
7                  CoClustering   GoogleLocalReviewsReader       100
8                  CoClustering              GowallaReader       100
9                  CoClustering              Jester2Reader       100
10                 CoClustering               LastFMReader       100
11                 CoClustering     MarketBiasAmazonReader       100
12                 CoClustering   MarketBiasModClothReader       100
13                 CoClustering   

In [5]:
# just to be sure, create a column that joins the algorithm and dataset name
df_expt.loc[:, "alg_data_name"] = df_expt[["alg_name", "dataset_name"]].agg('-'.join, axis=1)

/Users/duncan/miniconda3/envs/recsys/lib/python3.6/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/Users/duncan/miniconda3/envs/recsys/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [41]:
# if there are any rows with no metrics, drop them...
metric_col_list = [c for c in df_expt.columns if "_metric_" in c]

df_expt.loc[:, "all_na_metrics"] = df_expt[metric_col_list].isna().all(axis=1)

# drop cols with all na metrics
df_expt = df_expt.loc[~df_expt["all_na_metrics"], :]

/Users/duncan/miniconda3/envs/recsys/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [44]:
# now find the max and min metric for each alg-dataset combo (just in case there are any min metrics...)
metric_col_list = [c for c in df_expt.columns if "_metric_" in c]

max_metric_df = df_expt.groupby(["alg_name", "dataset_name"])[metric_col_list].max().reset_index()
max_metric_df.rename(columns={
    c: "max_" + c
    for c in metric_col_list
}, inplace=True)

min_metric_df = df_expt.groupby(["alg_name", "dataset_name"])[metric_col_list].min().reset_index()
min_metric_df.rename(columns={
    c: "min_" + c
    for c in metric_col_list
}, inplace=True)


In [45]:
min_metric_df

,alg_name,dataset_name,min_test_metric_ARHR_ALL_HITS_cut_1,min_test_metric_ARHR_ALL_HITS_cut_10,min_test_metric_ARHR_ALL_HITS_cut_15,min_test_metric_ARHR_ALL_HITS_cut_2,min_test_metric_ARHR_ALL_HITS_cut_20,min_test_metric_ARHR_ALL_HITS_cut_3,min_test_metric_ARHR_ALL_HITS_cut_30,min_test_metric_ARHR_ALL_HITS_cut_4,...,min_test_metric_USERS_IN_GT_cut_3,min_test_metric_USERS_IN_GT_cut_30,min_test_metric_USERS_IN_GT_cut_4,min_test_metric_USERS_IN_GT_cut_40,min_test_metric_USERS_IN_GT_cut_5,min_test_metric_USERS_IN_GT_cut_50,min_test_metric_USERS_IN_GT_cut_6,min_test_metric_USERS_IN_GT_cut_7,min_test_metric_USERS_IN_GT_cut_8,min_test_metric_USERS_IN_GT_cut_9
0,CoClustering,AnimeReader,0.000000,0.000092,0.000288,0.000007,0.000522,0.000007,0.000879,0.000011,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,CoClustering,BookCrossingReader,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,CoClustering,CiaoDVDReader,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
3,CoClustering,DatingReader,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4,CoClustering,EpinionsReader,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
5,CoClustering,FilmTrustReader,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
6,CoClustering,FrappeReader,0.000000,0.000400,0.000492,0.000000,0.000492,0.000000,0.000492,0.000000,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
7,CoClustering,GowallaReader,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
8,CoClustering,Jester2Reader,0.003373,0.016414,0.018999,0.007056,0.021155,0.009287,0.023918,0.011027,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
9,CoClustering,LastFMReader,0.000000,0.000053,0.000053,0.000000,0.000053,0.000000,0.000053,0.000000,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [53]:
# merge the min and max metric dataframes
df_hpo = max_metric_df.merge(min_metric_df, on=["alg_name", "dataset_name"], how="inner")
df_hpo.head()

,alg_name,dataset_name,max_test_metric_ARHR_ALL_HITS_cut_1,max_test_metric_ARHR_ALL_HITS_cut_10,max_test_metric_ARHR_ALL_HITS_cut_15,max_test_metric_ARHR_ALL_HITS_cut_2,max_test_metric_ARHR_ALL_HITS_cut_20,max_test_metric_ARHR_ALL_HITS_cut_3,max_test_metric_ARHR_ALL_HITS_cut_30,max_test_metric_ARHR_ALL_HITS_cut_4,...,min_test_metric_USERS_IN_GT_cut_3,min_test_metric_USERS_IN_GT_cut_30,min_test_metric_USERS_IN_GT_cut_4,min_test_metric_USERS_IN_GT_cut_40,min_test_metric_USERS_IN_GT_cut_5,min_test_metric_USERS_IN_GT_cut_50,min_test_metric_USERS_IN_GT_cut_6,min_test_metric_USERS_IN_GT_cut_7,min_test_metric_USERS_IN_GT_cut_8,min_test_metric_USERS_IN_GT_cut_9
0,CoClustering,AnimeReader,0.000043,4.104161e-04,0.000673,0.000101,0.000898,0.000134,0.001251,0.000174,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,CoClustering,BookCrossingReader,0.000000,2.363228e-05,0.000024,0.000000,0.000024,0.000015,0.000025,0.000015,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,CoClustering,CiaoDVDReader,0.000211,3.162555e-04,0.000316,0.000316,0.000327,0.000316,0.000336,0.000316,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
3,CoClustering,DatingReader,0.000000,8.208624e-07,0.000001,0.000000,0.000001,0.000000,0.000001,0.000000,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4,CoClustering,EpinionsReader,0.000000,8.775933e-06,0.000009,0.000000,0.000009,0.000000,0.000010,0.000009,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [60]:
df_hpo.to_csv("./performance_meta_dataset.csv")